In [12]:
import os

In [13]:
%pwd

'c:\\Users\\malis\\OneDrive\\Desktop\\Shrujan\\mlops\\End-to-end-Machine-Learning-project-with-mlflow'

In [14]:
os.chdir("../")
%pwd

'c:\\Users\\malis\\OneDrive\\Desktop\\Shrujan\\mlops'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ShrujanMali"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f8e93b3096836cd7efec776ab771847ce8599a5b"

# set MLFLOW_TRACKING_URI="https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow"
# set MLFLOW_TRACKING_USERNAME="ShrujanMali"
# set MLFLOW_TRACKING_PASSWORD="f8e93b3096836cd7efec776ab771847ce8599a5b"

# set MLFLOW_TRACKING_URI="https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow"
# set MLFLOW_TRACKING_USERNAME=<username>
# set MLFLOW_TRACKING_PASSWORD="password"

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [17]:
os.chdir("c:\\Users\\malis\\OneDrive\\Desktop\\Shrujan\\mlops\\End-to-end-Machine-Learning-project-with-mlflow\\src")
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri= "https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow"
        )
        return model_evaluation_config

In [22]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject.utils.common import save_json
print(mlflow.get_tracking_uri())



https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow


In [23]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

print(mlflow.get_tracking_uri())


https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow


In [ ]:
os.chdir("c:\\Users\\malis\\OneDrive\\Desktop\\Shrujan\\mlops\\End-to-end-Machine-Learning-project-with-mlflow")
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

print("4", mlflow.get_tracking_uri())

[2025-03-23 23:39:27,818: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-23 23:39:27,823: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-23 23:39:27,830: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-23 23:39:27,834: INFO: common: created directory at: artifacts]
[2025-03-23 23:39:27,838: INFO: common: created directory at: artifacts/model_evaluation]
[2025-03-23 23:39:27,943: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow/api/2.0/mlflow/runs/create]
[2025-03-23 23:39:28,647: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\malis\anaconda3\envs\envname\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\malis\anaconda3\envs\envname\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/03/23 23:39:42 INFO mlflow.store.mo

https://dagshub.com/ShrujanMali/End-to-end-Machine-Learning-project-with-mlflow.mlflow
